In [1]:
import sys
sys.path.append('..')

from solver import solve
from vec import Vec
from mat import Mat
from matutil import rowdict2mat, mat2rowdict

In [21]:
def simplex_step(A, b, c, R_square, show_bases=False):
    if show_bases: print("basis: ", R_square)
    R = A.D[0]
    # Extract the subsystem
    A_square = Mat((R_square, A.D[1]), {(r, c): A[r, c] for r,c in A.f if r in R_square})
    b_square = Vec(R_square, {k: b[k] for k in R_square})
    # Compute the current vertex
    x = solve(A_square, b_square)
    print("x: \n", x)
    print("(value: ",c*x,") ")
    # Compute a possibly feasible dual solution
    y_square = solve(A_square.transpose(), c) #compute entries with labels in R_square
    y = Vec(R, y_square.f) #Put in zero for the other entries
    print("y: \n", y)
    if min(y.f.values()) >= -1e-10: return ('OPTIMUM', x) #found optimum!
    R_leave = {i for i in R if y[i] < -1e-10} #labels at which y is negative
    r_leave = min(R_leave, key=hash) #choose first label at which y is negative
    # Compute the direction to move
    d = Vec(R_square, {r_leave: 1})
    w = solve(A_square, d)
    print("w: \n", w)
    # Compute how far to move
    Aw = A*w # compute once because we use it many times
    print("A*w: \n", Aw)
    R_enter = {r for r in R if Aw[r] < -1e-10}
    if len(R_enter) == 0: return ('UNBOUNDED', None)
    Ax = A*x # compute once because we use it many times
    delta_dict = {r: (b[r] - Ax[r])/(Aw[r]) for r in R_enter}
    print("delta_dict: \n", delta_dict)
    delta = min(delta_dict.values())
    # Compute the new tight constraint
    r_enter = min({r for r in R_enter if delta_dict[r] == delta}, key=hash)[0]
    # Update the set representing the basis
    R_square.discard(r_leave)
    R_square.add(r_enter)
    return ('STEP', None)

In [3]:
def optimize(A, b, c, R_square, show_basis=False):
    """ solve min { c*x : A*x >= b}
        starting vertex is specified by R_square, i.e.
        R_square is set of row-labels of subsystem specifying initial vertex
        At the end, if the optimization is successful,
        R_square is the set of row-labels specifying final (i.e., optimal) vertex.
        After each pivot step but the last, a - is printed.
    """
    i = 0
    while True:
        i = i+1
        outcome, answer = simplex_step(A, b, c, R_square, show_basis)
        if outcome == 'STEP':
            print(i, " ", end='')
            sys.stdout.flush()
            continue
        if outcome == 'UNBOUNDED':
            return None
        assert outcome == 'OPTIMUM'
        return answer

In [4]:
def dict_union(*args):
    dict = {}
    for d in args:
        dict.update(d)
    return dict

In [33]:
def find_vertex(A, b, R_square):
    assert len(A.D[1]) == len(R_square)
    def new_name(r): return (r, -1)
    A_square = Mat((R_square, A.D[1]), {(r,c): A[r,c] for r,c in A.f if r in R_square})
    b_square = Vec(R_square, {k: b[k] for k in R_square})
    x = solve(A_square, b_square)
    A_x = A*x
    missing = A.D[0].difference(R_square) # set of row-labels not in R_square
    extra = {new_name(r) for r in missing}
    f = dict_union(A.f, {(r, new_name(r)): 1 for r in missing}, {(e, e): 1 for e in extra})
    A_with_extra = Mat((A.D[0].union(extra), A.D[1].union(extra)), f)
    b_with_extra = Vec(b.D.union(extra), b.f)
    new_R_square = R_square | {r if A_x[r] - b[r] < -1e-10 else new_name(r) for r in missing}
    print("new_R_square: \n", new_R_square)
    c = Vec(A.D[1].union(extra), {e: 1 for e in extra})
    answer = optimize(A_with_extra, b_with_extra, c, new_R_square)
    if answer is None: return False
    basis_candidates = list(new_R_square | A.D[0])
    print("basis_candidates: \n", basis_candidates)
    R_square.clear()
    n = len(A.D[1])
    R_square.update(set(basis_candidates[:n]))
    return True

In [35]:
# 13.8.4

C = {'rice', 'lard'}
R = {'C', 'D', 'E', 'lard-nonneg', 'rice-nonneg'}

A = Mat((R,C), {
        ('C', 'rice'): 2.0, ('C', 'lard'): 10.0,
        ('D', 'rice'): 2.0, ('D', 'lard'): 1.0,
        ('E', 'rice'): 8.0, ('E', 'lard'): 1.0,
        ('lard-nonneg', 'rice'): 0.0, ('lard-nonneg', 'lard'): 1.0, 
        ('rice-nonneg', 'rice'): 1.0, ('rice-nonneg', 'lard'): 0.0
    })

b = Vec(R, {
        'C': 5.0,
        'D': 1.0,
        'E': 2.0,
        'lard-nonneg': 0.0,
        'rice-nonneg': 0.0
    })

c = Vec(C, {'rice': 1.0, 'lard': 1.7})

R_square = {'E', 'rice-nonneg'}

#print("vertex found?", find_vertex(A, b, R_square))

print("R_square: \n", R_square)

optimize(A, b, c, R_square, show_basis=True)

R_square: 
 {'rice-nonneg', 'E'}
basis:  {'rice-nonneg', 'E'}
x: 
 
 lard rice
----------
    2    0
(value:  3.4 ) 
y: 
 
 C D   E lard-nonneg rice-nonneg
--------------------------------
 0 0 1.7           0       -12.6
w: 
 
 lard rice
----------
   -8    1
A*w: 
 
   C  D E lard-nonneg rice-nonneg
---------------------------------
 -78 -6 0          -8           1
delta_dict: 
 {'lard-nonneg': 0.25, 'C': 0.19230769230769232, 'D': 0.16666666666666666}
1  basis:  {'D', 'E'}
x: 
 
  lard  rice
------------
 0.667 0.167
(value:  1.3000000000000003 ) 
y: 
 
 C   D    E lard-nonneg rice-nonneg
-----------------------------------
 0 2.1 -0.4           0           0
w: 
 
   lard  rice
-------------
 -0.333 0.167
A*w: 
 
  C        D E lard-nonneg rice-nonneg
--------------------------------------
 -3 5.55E-17 1      -0.333       0.167
delta_dict: 
 {'lard-nonneg': 2.000000000000001, 'C': 0.6666666666666672}
2  basis:  {'D', 'C'}
x: 
 
  lard  rice
------------
 0.444 0.278
(value:  1.0333

Vec({'lard', 'rice'},{'lard': 0.44444444444444453, 'rice': 0.2777777777777775})

13.13

### 線形計画とは何か?

線形不等式$f(x_1, \dots, x_n) = a_1x_1 + \dots a_nx_n \ge b$の集合を行列を用いて以下のように表現する。

$Ax \ge b$

目的関数$g(x_1, \dots, x_n) = c_1x_1 + \dots c_nx_n$をベクトルを用いて以下のように表現する。

$c\cdot x$

線形計画は$Ax \ge b$という制約条件のもと$\min c\cdot x$を解く問題である。



### 資源配分問題はどのようにして線形計画として定式化されるか?

不等式$Ax \ge b$のxを各資源の量、Aを資源の価値（栄養価など）、bを最低限必要な価値とし、目的関数$c\cdot x$のcを各資源の費用を表すようにすれば、資源配分問題を線形計画として表せる。

### 線形計画は異なる複数の形式で表すことができるが、それはどのようなものか?

$\min \{c \cdot x: Ax \ge b\}$は$c\_ = -c$とおくことで$\max \{c_ \cdot x: Ax \ge b\}$に置き換えられる。

$\min \{c \cdot x: Ax \ge b\}$は$A\_ = -A, b\_ = -b$とおくことで$\min \{c \cdot x: A\_x \le b\_\}$に置き換えられる。

等式による制約$a \cdot x = b$は$a \cdot x \ge b$かつ$a \cdot x \le b$($-a \cdot x \ge -b$)とすることで表現できる

### 線形計画法における双対性とは何か?

主線系計画$\min \{ c \cdot x: Ax \ge b \}$に対して$\max \{ b \cdot y: y^\top A = c, y \ge 0\}$をその双対という。

目的関数の値$c\cdot \hat{x}$と$\hat{y}\cdot b$の主実行可能解$\hat{x}$と双対実行可能解$\hat{y}$がが等しい場合、これらの解は最適解である。

### シンプレックスアルゴリズムの基底(おお!)とは何か?

多面体の頂点？

多面体$P = \{ x: Ax \ge b\}$上のベクトルvに対し、$Ax \ge b$の部分系$A_\square x \ge b_\square$で、vが$A_\square x = b_\square$の唯一の解となるものが存在するとき、vをPの頂点という。

### 線形計画法とプレイヤーが 2 人のゼロサムゲームはどのように関連するか?


